# Project: Machine Learning

**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
   
2. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive if you haven't already. This will be the repository for all your completed assignment and project files, aiding you in keeping your work organized and accessible.
   
3. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
4. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
   
5. **Setting Permission toPublic**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.

## Student Identity

In [1]:
# @title #### Student Identity
student_id = "REA110FJ" # @param {type:"string"}
name = "Citra Handan" # @param {type:"string"}
drive_link = "https://colab.research.google.com/drive/1JmrfxZ7ceYe3fa2D2wBCaYegFVcuAk-g"  # @param {type:"string"}

assignment_id = "00_ml_project"

# Import grader package
!pip install rggrader
from rggrader import submit, submit_image

## Project Description

In this Machine Learning Project, you will create your own supervised Machine Learning (ML) model. We will use the full FIFA21 Dataset and we will identify players that are above average.

We will use the column "Overall" with a treshold of 75 to define players that are 'Valuable'. This will become our target output which we need for a supervised ML model. Because we use the "Overall" as our target output, you cannot use "Overall" in your features, this will be explained further below.

This project will provide a comprehensive overview of your abilities in machine learning, from understanding the problem, choosing the right model, training, and optimizing it.

## Grading Criteria

Your score will be awarded based on the following criteria:
* 100: The model has an accuracy of more than 80% and an F1 score of more than 85%. This model is excellent and demonstrates a strong understanding of the task.
* 90: The model has an accuracy of more than 75% and an F1 score of more than 80%. This model is very good, with some room for improvement.
* 80: The model has an accuracy of more than 70% and an F1 score between 70% and 80%. This model is fairly good but needs improvement in balancing precision and recall.
* 70: The model has an accuracy of more than 65% and an F1 score between 60% and 70%. This model is below average and needs significant improvement.
* 60 or below: The model has an accuracy of less than 65% or an F1 score of less than 60%, or the student did not submit the accuracy and F1 score. This model is poor and needs considerable improvement.

Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Happy modeling!

>Note: If you get the accuracy of 100% and F1 score of 100%, while it may earn you good grades, it's an indication of overfitting.

In [2]:
# Write any package/module installation that you need
# pip install goes here, this helps declutter your output below
# Import package
!pip install xgboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

## Load the dataset and clean it

In this task, you will prepare and load your dataset. You need to download the full FIFA 21 Dataset from the link here: [Kaggle FIFA Player Stats Database](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?resource=download&select=FIFA21_official_data.csv).

>Note: Make sure you download FIFA 21 dataset.
>
>![FIFA21 Dataset](https://storage.googleapis.com/rg-ai-bootcamp/projects/fifa21_dataset-min.png)

After you download the dataset, you will then import the dataset then you will clean the data. For example there may be some empty cell in the dataset which you need to fill. Maybe there are also data that you need to convert to numeric value for analysis. Identify the data that is incomplete and fix them.

In the code block below, you can use the comments to guide you on what to do.

In [4]:
# Write your preprocessing and data cleaning here

# Load your data
df_FIFA21 = pd.read_csv("/content/FIFA21_official_data.csv")
df_FIFA21.head()
# Check your data for empty cell

# Assuming df_FIFA21 is your DataFrame
nonobject_columns = df_FIFA21.select_dtypes(exclude=['object'])

# Check null data
df_null = nonobject_columns.isnull().sum()[nonobject_columns.isnull().sum() > 0]

# Fill the empty cell with data or drop the column
df_FIFA21['Jersey Number'] = df_FIFA21['Jersey Number'].fillna(0).astype(float)
df_FIFA21['Volleys'] = df_FIFA21['Volleys'].fillna(0).astype(float)
df_FIFA21['Curve'] = df_FIFA21['Curve'].fillna(0).astype(float)
df_FIFA21['Agility'] = df_FIFA21['Agility'].fillna(0).astype(float)
df_FIFA21['Balance'] = df_FIFA21['Balance'].fillna(0).astype(float)
df_FIFA21['Jumping'] = df_FIFA21['Jumping'].fillna(0).astype(float)
df_FIFA21['Interceptions'] = df_FIFA21['Interceptions'].fillna(0).astype(float)
df_FIFA21['Positioning'] = df_FIFA21['Positioning'].fillna(0).astype(float)
df_FIFA21['Vision'] = df_FIFA21['Vision'].fillna(0).astype(float)
df_FIFA21['Composure'] = df_FIFA21['Composure'].fillna(0).astype(float)
df_FIFA21['SlidingTackle'] = df_FIFA21['SlidingTackle'].fillna(0).astype(float)
df_FIFA21['DefensiveAwareness'] = df_FIFA21['DefensiveAwareness'].fillna(0).astype(float)
df_FIFA21.drop(columns=["Marking"], inplace=True)

df_FIFA21.drop(columns=["Name"], inplace=True)
df_FIFA21.drop(columns=["Photo"], inplace=True)
df_FIFA21.drop(columns=["Nationality"], inplace=True)
df_FIFA21.drop(columns=["Flag"], inplace=True)
df_FIFA21.drop(columns=["Club"], inplace=True)
df_FIFA21.drop(columns=["Club Logo"], inplace=True)
df_FIFA21.drop(columns=["Body Type"], inplace=True)
df_FIFA21.drop(columns=["Real Face"], inplace=True)
df_FIFA21.drop(columns=["Position"], inplace=True)
df_FIFA21.drop(columns=["Joined"], inplace=True)
df_FIFA21.drop(columns=["Contract Valid Until"], inplace=True)
df_FIFA21.drop(columns=["Height"], inplace=True)
df_FIFA21.drop(columns=["Weight"], inplace=True)
df_FIFA21.drop(columns=["Loaned From"], inplace=True)

columns_to_process = ['Value','Wage','Release Clause']

for i in columns_to_process:
    df_FIFA21[i] = df_FIFA21[i].replace({'€': '', 'K': 'e3', 'M': 'e6'}, regex=True)
    df_FIFA21[i] = df_FIFA21[i].apply(pd.to_numeric, errors='coerce')

df_FIFA21.loc[:,['Value','Wage','Release Clause']]

# Data Preprocessing
lb = preprocessing.LabelEncoder()

df_FIFA21["Preferred Foot"] = lb.fit_transform(df_FIFA21[["Preferred Foot"]])
df_FIFA21["Work Rate"] = lb.fit_transform(df_FIFA21[["Work Rate"]])
df_FIFA21["Best Position"] = lb.fit_transform(df_FIFA21[["Best Position"]])
df_FIFA21.loc[:,['Preferred Foot','Work Rate','Best Position']]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Preferred Foot,Work Rate,Best Position
0,1,2,14
1,1,0,0
2,1,0,0
3,0,8,14
4,0,0,6
...,...,...,...
17103,1,8,5
17104,1,8,5
17105,1,8,5
17106,1,8,5


In [6]:
# Choose the features that has high correlation with our target "Overall"
correlation_matrix = df_FIFA21.corr()
high_correlation_mask = correlation_matrix >= 0.5
high_correlation_mask

,ID,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
ID,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Age,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Overall,False,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
Potential,False,False,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
Value,False,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
Wage,False,False,True,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
Special,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
Preferred Foot,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
International Reputation,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
Weak Foot,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


## Build and Train your model

In this task you will analyze the data and select the features that is best at predicting if the Player is a 'Valuable' player or not.

The first step is to **define the target output** that you will use for training. Here's an example of how to create a target output:
- `df['OK Player'] = df['Overall'].apply(lambda x: 1 if x >= 50 else 0) #Define the OK Player using treshold of 50.`

Next you will **identify the features** that will best predict a 'Valuable' player. You are required to **submit the features you selected** in the Submission section below. Because we use the "Overall" as our target output, the use of "Overall" in your features is not allowed. You will automatically get 0 if you submit "Overall" in your features.

Once you identify the features, you will then **split the data** into Training set and Testing/Validation set.

Depending on the features you selected, **you may need to scale the features**.

Now you will **train your model, choose the algorithm** you are going to use carefully to make sure it gives the best result.

Once you have trained your model, you need to test the model effectiveness. **Make predictions against your Testing/Validation set** and evaluate your model. You are required to **submit the Accuracy Score and F1 score** in the Submission section below.

In the code block below, you can use the comments to guide you on what to do.

We have also provided 3 variables that you must use in your code, `ml_features`, `ml_accuracy` and `ml_f1_score`. You can move the variables around your code, assign values to them, but you cannot delete them.

In [13]:
# Write your code here

# Define the target output (Good >= 75)
y = df_FIFA21['Overall'].apply(lambda x: 1 if x >= 50 else 0)

X = df_FIFA21.loc[:,["Age","Value","Wage","Potential","Special","ShortPassing",
                     "Reactions","ShotPower","Composure","Release Clause"]]
# Identify the features you will use in your model
ml_features = ["Age","Value","Wage","Potential","Special","ShortPassing",
                     "Reactions","ShotPower","Composure","Release Clause"]

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42, stratify=y)

# Scale the features (if needed, optional)


# Train the model
xgb_clf = xgb.XGBClassifier(max_depth=5, n_estimators=1000, learning_rate=0.3,
                            n_jobs=-1)
xgb_clf.fit(X_train, y_train)

# Make predictions using the test set
print('clasification report')
print(classification_report(y_test, xgb_clf.predict(X_test)))
print('confusion matrix')
print(confusion_matrix(y_test, xgb_clf.predict(X_test)))
print('f1_score')
print(f1_score(y_test, xgb_clf.predict(X_test)))
print('accuracy score')
print(accuracy_score(y_test, xgb_clf.predict(X_test)))

# Evaluate the model
ml_accuracy = accuracy_score(y_test, xgb_clf.predict(X_test))
ml_f1_score = f1_score(y_test, xgb_clf.predict(X_test))



clasification report
              precision    recall  f1-score   support

           0       0.68      0.46      0.55        28
           1       1.00      1.00      1.00      5105

    accuracy                           1.00      5133
   macro avg       0.84      0.73      0.78      5133
weighted avg       1.00      1.00      1.00      5133

confusion matrix
[[  13   15]
 [   6 5099]]
f1_score
0.997945004403562
accuracy score
0.9959088252483927


## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [14]:
# Submit Method

# Do not change the code below
question_id = "01_ml_project_features"
submit(student_id, name, assignment_id, str(ml_features), question_id, drive_link)
question_id = "02_ml_project_accuracy"
submit(student_id, name, assignment_id, str(ml_accuracy), question_id, drive_link)
question_id = "03_ml_project_f1score"
submit(student_id, name, assignment_id, str(ml_f1_score), question_id, drive_link)

'Assignment successfully submitted'

## FIN